In [474]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

In [300]:
training=pd.read_csv("training.csv")

In [301]:
X_train=training.iloc[:,4:16]

In [302]:
X_train

,Played Value H,Played Value A,Total Value H,Total Value A,Attack H,Midfield H,Defense H,Goalie H,Attack A,Midfield A,Defense A,Goalie A
0,50400000.0,46575000.0,96458000,141390000,16650000.0,4950000.0,25200000.0,3600000.0,10350000.0,12600000.0,15525000.0,8100000.0
1,61650000.0,60840000.0,205583000,110880000,33750000.0,9000000.0,15300000.0,3600000.0,35100000.0,8550000.0,13590000.0,3600000.0
2,105750000.0,328500000.0,280125000,489285000,22500000.0,53550000.0,27900000.0,1800000.0,76500000.0,99000000.0,131400000.0,21600000.0
3,43650000.0,99000000.0,122265000,196785000,15750000.0,14850000.0,12150000.0,900000.0,63900000.0,10800000.0,15300000.0,9000000.0
4,263250000.0,37575000.0,546030000,118800000,109800000.0,101700000.0,50400000.0,1350000.0,12375000.0,4500000.0,18450000.0,2250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
744,87750000.0,403200000.0,187448000,795830000,24300000.0,18000000.0,41850000.0,3600000.0,115200000.0,130500000.0,103500000.0,54000000.0
745,119250000.0,516150000.0,267300000,908100000,38700000.0,19800000.0,58500000.0,2250000.0,207000000.0,198000000.0,108000000.0,3150000.0
746,58950000.0,105300000.0,148941000,173205000,24300000.0,9000000.0,21150000.0,4500000.0,23400000.0,29700000.0,32400000.0,19800000.0
747,417600000.0,156600000.0,746640000,273825000,178200000.0,115200000.0,101700000.0,22500000.0,94500000.0,27000000.0,34200000.0,900000.0


In [303]:
X=X_train

In [304]:
y_train=training.iloc[:,18]

In [305]:
testing=pd.read_csv('test.csv')

In [306]:
X_test=testing.iloc[:,4:16]

In [307]:
X_t=X_test

In [308]:
y_test=testing.iloc[:,18]

In [309]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [310]:
X_test=scaler.transform(X_test)
X_train=scaler.transform(X_train)

In [311]:
#KNN

In [347]:
from sklearn.neighbors import KNeighborsClassifier

In [348]:
np.sqrt(len(X_train))

27.367864366808018

In [373]:
knnscores=[]
for i in range(3,50):
    knn_clf=KNeighborsClassifier(n_neighbors=i).fit(X_train,y_train)
    y_pred=knn_clf.predict(X_test)
    knnscores.append(accuracy_score(y_test,y_pred))
    

In [376]:
#Baseline (Default k = 5 ) score= Accuracy: 0.5394736842105263
max(knnscores)

0.5868421052631579

In [377]:
knnscores.index(max(knnscores))+3

23

In [378]:
knn_clf=KNeighborsClassifier(n_neighbors=23).fit(X_train,y_train)

In [465]:
y_pred_knn=knn_clf.predict(X_test)

In [467]:
labels = np.unique(y_test)
result_knn = confusion_matrix(y_test, y_pred_knn)
a =  confusion_matrix(y_test, y_pred_knn, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,86,12,30
D,20,5,46
H,33,16,132


In [472]:
score_knn = accuracy_score(y_test,y_pred_knn)
print("Accuracy:",score_knn)

Accuracy: 0.5868421052631579


In [475]:
param_grid_knn={
    'n_neighbors':range(2,40),
    'weights':['uniform','distance'],
    'metric':['euclidean','manhattan','minkowski','chebyshev']
}

In [477]:
gs_knn=GridSearchCV(KNeighborsClassifier(),param_grid=param_grid_knn)

In [478]:
gs_knn.fit(X_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski',
                                    'chebyshev'],
                         'n_neighbors': range(2, 40),
                         'weights': ['uniform', 'distance']})

In [480]:
gs_knn.best_estimator_

KNeighborsClassifier(metric='chebyshev', n_neighbors=24, weights='distance')

In [481]:
y_pred_knn_gs=gs_knn.predict(X_test)

In [487]:
result_knn_gs = accuracy_score(y_test,y_pred_knn_gs)
print("Accuracy:",result_knn_gs)

Accuracy: 0.5657894736842105


In [488]:
#Best from KNN: K=23, other params at default
#Accuracy: 0.5868421052631579

In [317]:
#SVM

In [318]:
from sklearn import svm

In [319]:
linear = svm.SVC(kernel='linear', C=.01, decision_function_shape='ovo').fit(X_train, y_train)

In [320]:
y_pred_linear=linear.predict(X_test)

In [493]:
a =  confusion_matrix(y_test, y_pred_linear, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,70,0,58
D,15,0,56
H,28,0,153


In [494]:
result_svm_linear = accuracy_score(y_test,y_pred_linear)
print("Accuracy:",result2)

Accuracy: 0.5868421052631579


In [323]:
rbf = svm.SVC(kernel='rbf',  decision_function_shape='ovo').fit(X_train, y_train)
poly = svm.SVC(kernel='poly', decision_function_shape='ovo').fit(X_train, y_train)
sig = svm.SVC(kernel='sigmoid',  decision_function_shape='ovo').fit(X_train, y_train)

In [324]:
y_pred_rbf=rbf.predict(X_test)
y_pred_poly=poly.predict(X_test)
y_pred_sig=sig.predict(X_test)


In [495]:
result_svm_rbf = accuracy_score(y_test,y_pred_rbf)
print("Accuracy:",result_svm_rbf)

Accuracy: 0.5894736842105263


In [496]:
result_svm_poly = accuracy_score(y_test,y_pred_poly)
print("Accuracy:",result_svm_poly)

Accuracy: 0.5473684210526316


In [497]:
result_svm_sig = accuracy_score(y_test,y_pred_sig)
print("Accuracy:",result_svm_poly)

Accuracy: 0.5473684210526316


In [498]:
a =  confusion_matrix(y_test, y_pred_rbf, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,70,0,58
D,15,0,56
H,27,0,154


In [501]:
a =  confusion_matrix(y_test, y_pred_sig, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,60,9,59
D,35,12,24
H,65,18,98


In [502]:
a =  confusion_matrix(y_test, y_pred_poly, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,54,5,69
D,7,4,60
H,25,6,150


In [383]:
#Hyperparameter Tuning for SVM

In [473]:
from sklearn.svm import SVC

In [606]:
param_grid={
    'kernel':['rbf'],
    'C' : np.linspace(.01,10,100),
    'gamma':np.linspace(.001,1,100)
}

In [402]:
gs_svc=GridSearchCV(SVC(decision_function_shape='ovo'),param_grid=param_grid)

In [403]:
gs_svc.fit(X_train,y_train)

GridSearchCV(estimator=SVC(decision_function_shape='ovo'),
             param_grid={'C': array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.71636364,  0.81727273,  0.91818182,
        1.01909091,  1.12      ,  1.22090909,  1.32181818,  1.42272727,
        1.52363636,  1.62454545,  1.72545455,  1.82636364,  1.92727273,
        2.02818182,  2.12909091,  2.23      ,  2.33090909,  2.43181818,
        2.53272727,  2...
       0.65690909, 0.667     , 0.67709091, 0.68718182, 0.69727273,
       0.70736364, 0.71745455, 0.72754545, 0.73763636, 0.74772727,
       0.75781818, 0.76790909, 0.778     , 0.78809091, 0.79818182,
       0.80827273, 0.81836364, 0.82845455, 0.83854545, 0.84863636,
       0.85872727, 0.86881818, 0.87890909, 0.889     , 0.89909091,
       0.90918182, 0.91927273, 0.92936364, 0.93945455, 0.94954545,
       0.95963636, 0.96972727, 0.97981818, 0.98990909, 1.        ]),
                         'kernel': ['rbf']})

In [405]:
svc_rbf=SVC(C=4.35,decision_function_shape='ovo',gamma=.001).fit(X_train,y_train)

In [406]:
y_pred_rbf=svc_rbf.predict(X_test)

In [409]:
result_rbf = accuracy_score(y_test,y_pred_rbf)
print("Accuracy:",result_rbf)

Accuracy: 0.5894736842105263


In [411]:
svc_rbf_lg=SVC(C=4.35,decision_function_shape='ovo',gamma=.0001).fit(X_train,y_train)

In [412]:
y_pred_rbf_lg=svc_rbf.predict(X_test)

In [413]:
result_rbf_lg = accuracy_score(y_test,y_pred_rbf_lg)
print("Accuracy:",result_rbf_lg)

Accuracy: 0.5894736842105263


In [415]:
param_grid2={
    'kernel':['linear'],
    'C' : np.linspace(.01,10,100),
    'gamma':np.linspace(.001,1,100)
}

In [416]:
gs_svc=GridSearchCV(SVC(decision_function_shape='ovo'),param_grid=param_grid2)

In [417]:
gs_svc.fit(X_train,y_train)

GridSearchCV(estimator=SVC(decision_function_shape='ovo'),
             param_grid={'C': array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.71636364,  0.81727273,  0.91818182,
        1.01909091,  1.12      ,  1.22090909,  1.32181818,  1.42272727,
        1.52363636,  1.62454545,  1.72545455,  1.82636364,  1.92727273,
        2.02818182,  2.12909091,  2.23      ,  2.33090909,  2.43181818,
        2.53272727,  2...
       0.65690909, 0.667     , 0.67709091, 0.68718182, 0.69727273,
       0.70736364, 0.71745455, 0.72754545, 0.73763636, 0.74772727,
       0.75781818, 0.76790909, 0.778     , 0.78809091, 0.79818182,
       0.80827273, 0.81836364, 0.82845455, 0.83854545, 0.84863636,
       0.85872727, 0.86881818, 0.87890909, 0.889     , 0.89909091,
       0.90918182, 0.91927273, 0.92936364, 0.93945455, 0.94954545,
       0.95963636, 0.96972727, 0.97981818, 0.98990909, 1.        ]),
                         'kernel': ['linear']})

In [419]:
gs_svc.best_estimator_

SVC(C=0.01, decision_function_shape='ovo', gamma=0.001, kernel='linear')

In [421]:
svc_linear_gs=SVC(C=.01,decision_function_shape='ovo',gamma=.001).fit(X_train,y_train)

In [422]:
y_pred_linear_gs=svc_linear_gs.predict(X_test)

In [489]:
result_linear_gs = accuracy_score(y_test,y_pred_linear_gs)
print("Accuracy:",result_linear_gs)

Accuracy: 0.4763157894736842


In [503]:
#SVM Best Performance:rbf, 
#Accuracy: 0.5894736842105263

In [426]:
#NN

In [331]:
from sklearn.neural_network import MLPClassifier

In [332]:
mlpclf=MLPClassifier(solver='lbfgs',random_state=1).fit(X_train, y_train)

In [333]:
y_pred_mlp=mlpclf.predict(X_test)

In [334]:
labels = np.unique(y_test)
result = confusion_matrix(y_test, y_pred_mlp)
a =  confusion_matrix(y_test, y_pred_mlp, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,47,37,44
D,23,16,32
H,35,49,97


In [427]:
result2 = accuracy_score(y_test,y_pred_mlp)
print("Accuracy:",result2)

Accuracy: 0.42105263157894735


In [429]:
mlpclf_tan=MLPClassifier(solver='lbfgs',random_state=1,activation='tanh').fit(X_train, y_train)

In [430]:
y_pred_tan=mlpclf_tan.predict(X_test)

In [432]:
result_tan = accuracy_score(y_test,y_pred_tan)
print("Accuracy:",result_tan)

Accuracy: 0.45


In [434]:
mlpclf_log=MLPClassifier(solver='lbfgs',random_state=1,activation='logistic').fit(X_train, y_train)

In [435]:
y_pred_log=mlpclf_log.predict(X_test)

In [437]:
result_log = accuracy_score(y_test,y_pred_log)
print("Accuracy:",result_log)

Accuracy: 0.5052631578947369


In [451]:
mlpclf_log=MLPClassifier(solver='lbfgs',random_state=1,activation='logistic',hidden_layer_sizes=(800,800)).fit(X_train, y_train)

In [504]:
y_pred_log=mlpclf_log.predict(X_test)
result_log = accuracy_score(y_test,y_pred_log)
print("Accuracy:",result_log)

Accuracy: 0.5684210526315789


In [ ]:
#NN Best perofrmance: logistic activation
#Accuracy: 0.5684210526315789

In [336]:
#ADABOOST

In [337]:
from sklearn.ensemble import AdaBoostClassifier

In [338]:
adaclf = AdaBoostClassifier(n_estimators=100, random_state=1).fit(X_train,y_train)

In [339]:
y_pred_ada=adaclf.predict(X_test)

In [340]:
labels = np.unique(y_test)
result = confusion_matrix(y_test, y_pred_ada)
a =  confusion_matrix(y_test, y_pred_ada, labels=labels)
pd.DataFrame(a, index=labels, columns=labels)

,A,D,H
A,66,27,35
D,20,20,31
H,38,39,104


In [341]:
result2 = accuracy_score(y_test,y_pred_ada)
print("Accuracy:",result2)

Accuracy: 0.5


In [454]:
adaclf_le = AdaBoostClassifier(n_estimators=1000, random_state=1).fit(X_train,y_train)

In [455]:
y_pred_ada_le=adaclf_le.predict(X_test)
result_adale = accuracy_score(y_test,y_pred_ada_le)
print("Accuracy:",result_adale)

Accuracy: 0.4868421052631579


In [519]:
Home=testing['HomeTeam']

In [520]:
Away=testing['AwayTeam']

In [521]:
rank={}
rank={k:0 for k in Home}

In [557]:
def ranker(inputPred):

    rank={k:0 for k in Home}
    for i in range(len(Home)):
        if inputPred[i]=='H':
            rank[Home[i]]+=3
        elif inputPred[i]=='A':
            rank[Away[i]]+=3
        else:
            rank[Home[i]]+=1
            rank[Away[i]]+=1
    rank1=dict(sorted(rank.items(), key=lambda item: item[1],reverse=True))
    return dict(zip(list(rank1.keys()),range(1,21)))

In [559]:
ranker(y_pred_log)

{'Liverpool': 1,
 'Manchester City': 2,
 'Tottenham Hotspur': 3,
 'Chelsea': 4,
 'Manchester United': 5,
 'Arsenal': 6,
 'Everton': 7,
 'Wolverhampton Wanderers': 8,
 'Leicester City': 9,
 'Fulham': 10,
 'West Ham United': 11,
 'AFC Bournemouth': 12,
 'Crystal Palace': 13,
 'Southampton': 14,
 'Burnley': 15,
 'Watford': 16,
 'Brighton and Hove Albion': 17,
 'Newcastle United': 18,
 'Huddersfield Town': 19,
 'Cardiff City': 20}

In [563]:
from sklearn.feature_selection import f_classif,SelectKBest

In [561]:
f_classif(X_train,y_train)

(array([41.60561754, 60.7243633 , 41.29510202, 57.98833829, 33.998498  ,
        36.75709647, 32.28828642, 14.53310059, 38.60669078, 60.68254951,
        47.18056027, 30.05303881]),
 array([7.40408407e-18, 3.68671946e-25, 9.79127550e-18, 3.90621029e-24,
        7.40077910e-15, 5.95683857e-16, 3.55961366e-14, 6.42999624e-07,
        1.11032334e-16, 3.82171141e-25, 5.07905300e-20, 2.80087494e-13]))

In [608]:
top_10_features = SelectKBest(f_classif, k=8).fit(X_train,y_train)

In [609]:
transformed_X=X_train[:,top_10_features.get_support()]

In [610]:
X_test_transformed=X_test[:,top_10_features.get_support()]

In [611]:
svc_linear_gs=SVC(kernel='rbf',decision_function_shape='ovo').fit(transformed_X,y_train)

In [612]:
y_pred_fs=svc_linear_gs.predict(X_test_transformed)

In [613]:
accuracy_score(y_test,y_pred_fs)

0.5657894736842105

In [614]:
param_grid={
    'kernel':['rbf'],
    'C' : np.linspace(.01,10,100),
    'gamma':np.linspace(.001,1,100)
}

In [615]:
gs_svc=GridSearchCV(SVC(decision_function_shape='ovo'),param_grid=param_grid)

In [617]:
gs_svc.fit(transformed_X,y_train)

GridSearchCV(estimator=SVC(decision_function_shape='ovo'),
             param_grid={'C': array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.71636364,  0.81727273,  0.91818182,
        1.01909091,  1.12      ,  1.22090909,  1.32181818,  1.42272727,
        1.52363636,  1.62454545,  1.72545455,  1.82636364,  1.92727273,
        2.02818182,  2.12909091,  2.23      ,  2.33090909,  2.43181818,
        2.53272727,  2...
       0.65690909, 0.667     , 0.67709091, 0.68718182, 0.69727273,
       0.70736364, 0.71745455, 0.72754545, 0.73763636, 0.74772727,
       0.75781818, 0.76790909, 0.778     , 0.78809091, 0.79818182,
       0.80827273, 0.81836364, 0.82845455, 0.83854545, 0.84863636,
       0.85872727, 0.86881818, 0.87890909, 0.889     , 0.89909091,
       0.90918182, 0.91927273, 0.92936364, 0.93945455, 0.94954545,
       0.95963636, 0.96972727, 0.97981818, 0.98990909, 1.        ]),
                         'kernel': ['rbf']})

In [ ]:
gs_svc